# Анализ данных студентов

In [162]:
import pandas as pd
import numpy as np

In [163]:
dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
}

# dataTrain = pd.read_csv("./train.csv", dtype=dtypes, nrows=10000) # Временно для быстроты работы
dataTrain = pd.read_csv("./train.csv", dtype=dtypes)

print("Train size:", dataTrain.shape)

Train size: (101230332, 10)


In [164]:
dataTrain.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,False,1,3,1,NaN,<NA>
1,1,56943,115,5716,False,2,2,1,37000.0,False
2,2,118363,115,128,False,0,0,1,55000.0,False
3,3,131167,115,7860,False,3,0,1,19000.0,False
4,4,137965,115,7922,False,4,1,1,11000.0,False


In [165]:
dataLectures = pd.read_csv("./lectures.csv")

print("Train size:", dataLectures.shape, dataLectures.head(), sep='\n')

Train size:
(418, 4)
   lecture_id  tag  part           type_of
0          89  159     5           concept
1         100   70     1           concept
2         185   45     6           concept
3         192   79     5  solving question
4         317  156     5  solving question


In [166]:
dataQuestions = pd.read_csv("./questions.csv")

print("Train size:", dataQuestions.shape)

Train size: (13523, 5)


In [167]:
dataQuestions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [168]:
dataTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int32  
 3   content_id                      int16  
 4   content_type_id                 boolean
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   answered_correctly              int8   
 8   prior_question_elapsed_time     float32
 9   prior_question_had_explanation  boolean
dtypes: boolean(2), float32(1), int16(2), int32(1), int64(2), int8(2)
memory usage: 3.2 GB


In [169]:
dataTrain.isnull().sum().to_frame().T

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,0,0,0,0,0,0,2351538,392506


In [170]:
# Получили сколько всего студентов проходило тестирования
countStudent = len(dataTrain['user_id'].unique()) 
countStudent

393656

In [171]:
print("Какие есть ответы", dataTrain['user_answer'].unique())
print("Варианты получения балов", dataTrain['answered_correctly'].unique())

Какие есть ответы [ 3  2  0  1 -1]
Варианты получения балов [ 1  0 -1]


In [172]:
# Объединение train.csv с questions.csv для вопросов (content_type_id == 0)
trainQuestions = pd.merge(dataTrain[dataTrain['content_type_id'] == 0], dataQuestions, left_on='content_id', right_on='question_id', how='left')

# Объединение train.csv с lectures.csv для лекций (content_type_id == 1)
trainLectures = pd.merge(dataTrain[dataTrain['content_type_id'] == 1], dataLectures, left_on='content_id', right_on='lecture_id', how='left')

# Объединение результатов
# Добавление столбца 'is_question' для различия между вопросами и лекциями
trainQuestions['is_question'] = True
trainLectures['is_question'] = False

# Объединение двух DataFrame
mergedData = pd.concat([trainQuestions, trainLectures], axis=0).reset_index(drop=True)

# Просмотр первых нескольких строк объединенного DataFrame
mergedData.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,question_id,bundle_id,correct_answer,part,tags,is_question,lecture_id,tag,type_of
0,0,0,115,5692,False,1,3,1,NaN,<NA>,5692.0,5692.0,3.0,5,151,True,NaN,NaN,NaN
1,1,56943,115,5716,False,2,2,1,37000.0,False,5716.0,5716.0,2.0,5,168,True,NaN,NaN,NaN
2,2,118363,115,128,False,0,0,1,55000.0,False,128.0,128.0,0.0,1,131 149 92,True,NaN,NaN,NaN
3,3,131167,115,7860,False,3,0,1,19000.0,False,7860.0,7860.0,0.0,1,131 104 81,True,NaN,NaN,NaN
4,4,137965,115,7922,False,4,1,1,11000.0,False,7922.0,7922.0,1.0,1,131 149 92,True,NaN,NaN,NaN


In [173]:
# Проанализируем общую статистику по правильным ответам
correct_answers = mergedData[mergedData['answered_correctly'] == 1].shape[0]
total_answers = mergedData[mergedData['content_type_id'] == 0].shape[0]
accuracy = correct_answers / total_answers
print(f"Процент правильных ответов: {accuracy * 100:.2f}%")

# Кластеризация вопросов по тегам
questionTags = mergedData['tags'].str.split().explode().value_counts()
print("Наиболее распространенные теги вопросов:")
print(questionTags.head(10))

Процент правильных ответов: 65.72%
Наиболее распространенные теги вопросов:
tags
92     18814335
38     16796328
81     13943138
29     12400546
162     9681952
102     8991751
143     7244646
136     5153159
131     4476973
62      3339768
Name: count, dtype: int64


## Статистика по студентам

In [174]:
# Фильтрация данных для вопросов (content_type_id == 0)
questions_data = mergedData[mergedData['content_type_id'] == 0]
# Фильтрация данных для лекций (content_type_id == 1)
lecture_data = mergedData[mergedData['content_type_id'] == 1]

In [175]:
# 1. Статистика по вопросам для каждого пользователя
# Группировка данных по user_id и агрегация
user_questions_stats = questions_data.groupby('user_id').agg(
    total_answers=('answered_correctly', 'count'),  # Общее количество ответов
    correct_answers=('answered_correctly', lambda x: (x == 1).sum()), # Количество правильных ответов
    prior_question_had_explanation=('prior_question_had_explanation', lambda x: (x == True).sum()),  # Количество просмотров своих ответов
    total_time_spent=('prior_question_elapsed_time', 'sum'),  # Общее время, затраченное на вопросы
).reset_index()
# Вычисление процента правильных ответов. Округление столбца 'accuracy' до целой части
user_questions_stats['accuracy'] = (user_questions_stats['correct_answers'] / user_questions_stats['total_answers'] * 100).round(0)
# Вычисление процента пользователей кто смотрел на свои ответы. Округление столбца 'checking_answers' до целой части
user_questions_stats['checking_answers'] = (user_questions_stats['prior_question_had_explanation'] / user_questions_stats['total_answers'] * 100).round(0)
# Вычисление среднего времени на вопрос
user_questions_stats['average_time_per_question'] = (user_questions_stats['total_time_spent'] / user_questions_stats['total_answers']).round(0)
# Преобразование среднего времени в часы и минуты
user_questions_stats['average_time_per_question'] = pd.to_timedelta(user_questions_stats['average_time_per_question'], unit='ms')
user_questions_stats['average_time_per_question'] = user_questions_stats['average_time_per_question'].apply(lambda x: f"{x.components.hours} ч {x.components.minutes} мин {x.components.seconds} сек")

# 2. Статистика по лекциям для каждого пользователя
# Группировка данных по user_id и агрегация
user_lectures_stats = lecture_data.groupby('user_id').agg(
    total_lectures=('content_id', 'count'),  # Общее количество просмотренных лекций
    total_parts=('part', 'nunique'),  # Количество уникальных категорий лекций
    total_tags=('tag', 'nunique'),  # Количество уникальных тегов лекций
    lecture_types=('type_of', 'unique')  # Уникальные типы лекций
).reset_index()

# Объединение статистики по вопросам и лекциям
user_stats = pd.merge(user_lectures_stats, user_questions_stats, on='user_id') # Объединение данных по user_id
# Сортировка секций по среднему времени ответа на вопрос
user_stats = user_stats.sort_values(by='average_time_per_question', ascending=False)
user_stats.head(10)

,user_id,total_lectures,total_parts,total_tags,lecture_types,total_answers,correct_answers,prior_question_had_explanation,total_time_spent,accuracy,checking_answers,average_time_per_question
15460,223266144,12,1,12,"[concept, solving question]",1439,797,1430,181732992.0,55.0,99.0,0 ч 2 мин 6 сек
70021,1006882723,5,1,3,"[concept, solving question]",132,63,121,16627950.0,48.0,92.0,0 ч 2 мин 5 сек
134744,1935387617,1,1,1,[concept],46,25,37,5736000.0,54.0,80.0,0 ч 2 мин 4 сек
142223,2042118658,6,2,6,"[concept, intention]",403,273,394,50261000.0,68.0,98.0,0 ч 2 мин 4 сек
53377,770124682,1,1,1,[concept],23,14,11,3384000.0,61.0,48.0,0 ч 2 мин 27 сек
94858,1364600348,2,1,2,[concept],244,168,205,29342476.0,69.0,84.0,0 ч 2 мин 0 сек
47903,691763599,10,4,8,"[concept, solving question]",365,179,328,25538660.0,49.0,90.0,0 ч 1 мин 9 сек
23317,336261965,12,2,11,"[intention, concept, solving question]",297,193,285,20583000.0,65.0,96.0,0 ч 1 мин 9 сек
5344,76250233,20,1,19,"[concept, solving question]",518,322,509,35756000.0,62.0,98.0,0 ч 1 мин 9 сек
30889,444957185,42,5,24,"[concept, solving question, intention]",1393,1008,1341,97208752.0,72.0,96.0,0 ч 1 мин 9 сек


## Статистика по вопросам

In [176]:

# 1. Сложность вопросов
# Группировка данных по question_id и агрегация
question_difficulty = questions_data.groupby('question_id').agg(
    total_answers=('answered_correctly', 'count'),  # Общее количество ответов
    correct_answers=('answered_correctly', lambda x: (x == 1).sum())  # Количество правильных ответов
).reset_index()

# Вычисление процента правильных ответов
question_difficulty['accuracy'] = (question_difficulty['correct_answers'] / question_difficulty['total_answers'] * 100).round()

question_difficulty.head(5)

,question_id,total_answers,correct_answers,accuracy
0,0.0,6903,6266,91.0
1,1.0,7398,6589,89.0
2,2.0,44905,24890,55.0
3,3.0,22973,17906,78.0
4,4.0,31736,19461,61.0


In [177]:
# 2. Сортировка вопросов по сложности (наименьший процент правильных ответов)
most_difficult_questions = question_difficulty.sort_values(by='accuracy', ascending=True)
most_difficult_questions.head(5)

,question_id,total_answers,correct_answers,accuracy
1485,1485.0,1,0,0.0
1486,1486.0,1,0,0.0
1484,1484.0,1,0,0.0
10007,10007.0,1,0,0.0
10062,10062.0,7444,683,9.0


In [178]:
# 3. Анализ секций теста
# Группировка данных по секциям и агрегация
part_difficulty = questions_data.groupby('part').agg(
    total_answers=('answered_correctly', 'count'),  # Общее количество ответов
    correct_answers=('answered_correctly', lambda x: (x == 1).sum())  # Количество правильных ответов
).reset_index()

# Вычисление процента правильных ответов
part_difficulty['accuracy'] = (part_difficulty['correct_answers'] / part_difficulty['total_answers'] * 100).round()

part_difficulty.head(5)

,part,total_answers,correct_answers,accuracy
0,1,7454570,5553896,75.0
1,2,18743404,13283339,71.0
2,3,8639907,6060514,70.0
3,4,8067676,5090684,63.0
4,5,40908153,24957570,61.0
